In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def classification(text):
    sequence_to_classify = text
    candidate_labels = ['Service', 'Measure', 'Objective', 'Remedy', 'Claim', 'Exception', 'Definition']
    return classifier(sequence_to_classify, candidate_labels, multi_label=True)

In [ ]:
sents_with_changes = []
for sent in list(docs[0].sents):
    sentence = str(sent)
    if "**" in sentence:
        sents_with_changes.append(sentence)

In [ ]:
i=1
dic = {'Service': [], 'Measure': [], 'Objective': [], 'Remedy': [], 'Claim': [], 'Exception': [], 'Definition': []}
dic_details = {}
for sentence in sents_with_changes:
  c_like = classification(sentence)
  category = c_like['labels'][c_like['scores'].index(max(c_like['scores']))]
  dic.get(category).append(sentence)
  dic_details[sentence] = [c_like['labels'], c_like['scores']]
  i = i+1
print(dic)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots() 
ax.plot(['Service', 'Measure', 'Objective', 'Remedy', 'Claim', 'Exception', 'Definition'], [len(dic['Service']), len(dic['Measure']), len(dic['Objective']), len(dic['Remedy']), len(dic['Claim']), len(dic['Exception']), len(dic['Definition'])]); 